# Persistence and Streaming

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
# here we are using in-memory database
# but we can use other external databases like postgres or even redis for caching

from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")

In [7]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        # here we are adding our checkpointer
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [13]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [14]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [15]:
# keep track of different threads inside checkpointer memory
thread = {"configurable": {"thread_id": "1"}}

In [16]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print("---->", v['messages'])

----> [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4ZqpjCM96QYQ9bvdh5TPY4sv', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 1158, 'total_tokens': 1181, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4d652bb1-da14-4cd4-93d3-a18d83b78eba-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4ZqpjCM96QYQ9bvdh5TPY4sv'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4ZqpjCM96QYQ9bvdh5TPY4sv'

In [17]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print("---->", v)

----> {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SVRevStL6uDCPIvizpODvHCC', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 2075, 'total_tokens': 2098, 'prompt_tokens_details': {'cached_tokens': 2048, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4299f931-0dbf-42d1-b835-8f191b026b6a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_SVRevStL6uDCPIvizpODvHCC'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_SVRevStL6uDCPI

In [18]:
# we can call with same thread_id and so it has access to full history 
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Los Angeles is currently warmer than San Francisco. The temperature in Los Angeles is 10.3°C (50.5°F), while in San Francisco, it is 8.9°C (48°F).', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 2776, 'total_tokens': 2820, 'prompt_tokens_details': {'cached_tokens': 2688, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'stop', 'logprobs': None}, id='run-09918845-b38b-4000-8330-3df234bc9ecc-0')]}


In [19]:
# let change the thread_id
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Could you please clarify what you're comparing to determine which is warmer? Are you comparing two specific locations, types of clothing, materials, or something else? Let me know so I can provide the appropriate information.", response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 149, 'total_tokens': 192, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'stop', 'logprobs': None}, id='run-c3daaf37-0f91-4a04-ba0b-cf49232c47c5-0')]}


## Streaming tokens

In [23]:
# astream_events is an async method, so we need to use async checkpointer  
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [24]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    print("Event Type:", kind)
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Event Type: on_chain_start
Event Type: on_chain_start
Event Type: on_chain_end
Event Type: on_chain_start
Event Type: on_chat_model_start
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_stream
Event Type: on_chat_model_end
Event Type: on_chain_start
Event Type: on_chain_end
Event Type: on_chain_start
Event Type: on_chain_end
Event Type: on_chain_start
Event Type: on_chain_end
Event Type: on_chain_stream
Event Type: on_chain_end
Event Type: on_chain_stream
Event Type: on_chain_start
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ELSmZ2F8EZGoY7UfTT9TT3cV'}
Event Type: on_tool_start
Back to the model!
Event Type: on_too

In [25]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

The| current| weather| in| San| Francisco| is| partly| cloudy| with| a| temperature| of| |48|.|0|°F| (|8|.|9|°C|).| The| wind| is| coming| from| the| southeast| at| |8|.|7| mph| (|14|.|0| k|ph|),| and| the| humidity| is| |89|%.| The| feels|-like| temperature| is| |44|.|0|°F| (|6|.|7|°C|).|